<a href="https://colab.research.google.com/github/ikfaisal/dsia/blob/master/DSIA_Lab_2_1_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.3 *Database Programming with Python and SQLite*

<a name="demo"></a>
# Using SQLite

## Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



## The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


## The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [14]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [15]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

OperationalError: table houses already exists

In [16]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

In [17]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

## Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [22]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)',last_sale)


# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [26]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

c.execute('INSERT INTO houses VALUES (?,?,?,?,?)', recent_sales[0])
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)', recent_sales[1])
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)', recent_sales[2])
# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
# 

conn.commit()

In [29]:
c.execute('SELECT * FROM houses').fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000)]

Select all rows from houses

## Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [31]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('../DATA/housing-data.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()

data

[[2104, 3, 70, 399900],
 [1600, 3, 28, 329900],
 [2400, 3, 44, 369000],
 [1416, 2, 49, 232000],
 [3000, 4, 75, 539900],
 [1985, 4, 61, 299900],
 [1534, 3, 12, 314900],
 [1427, 3, 57, 198999],
 [1380, 3, 14, 212000],
 [1494, 3, 15, 242500],
 [1940, 4, 7, 239999],
 [2000, 3, 27, 347000],
 [1890, 3, 45, 329999],
 [4478, 5, 49, 699900],
 [1268, 3, 58, 259900],
 [2300, 4, 77, 449900],
 [1320, 2, 62, 299900],
 [1236, 3, 78, 199900],
 [2609, 4, 5, 499998],
 [3031, 4, 21, 599000],
 [1767, 3, 44, 252900],
 [1888, 2, 79, 255000],
 [1604, 3, 13, 242900],
 [1962, 4, 53, 259900],
 [3890, 3, 36, 573900],
 [1100, 3, 60, 249900],
 [1458, 3, 29, 464500],
 [2526, 3, 13, 469000],
 [2200, 3, 28, 475000],
 [2637, 3, 25, 299900],
 [1839, 2, 40, 349900],
 [1000, 1, 5, 169900],
 [2040, 4, 75, 314900],
 [3137, 3, 67, 579900],
 [1811, 4, 24, 285900],
 [1437, 3, 50, 249900],
 [1239, 3, 22, 229900],
 [2132, 4, 28, 345000],
 [4215, 4, 66, 549000],
 [2162, 4, 43, 287000],
 [1664, 2, 40, 368500],
 [2238, 3, 37, 3299

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [32]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [33]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [34]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [35]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

## Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [36]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

In [37]:
c.execute('SELECT * FROM houses').fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000),
 (5, 2104, 3, 70, 399900),
 (6, 1600, 3, 28, 329900),
 (7, 2400, 3, 44, 369000),
 (8, 1416, 2, 49, 232000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (11, 1534, 3, 12, 314900),
 (12, 1427, 3, 57, 198999),
 (13, 1380, 3, 14, 212000),
 (14, 1494, 3, 15, 242500),
 (15, 1940, 4, 7, 239999),
 (16, 2000, 3, 27, 347000),
 (17, 1890, 3, 45, 329999),
 (18, 4478, 5, 49, 699900),
 (19, 1268, 3, 58, 259900),
 (20, 2300, 4, 77, 449900),
 (21, 1320, 2, 62, 299900),
 (22, 1236, 3, 78, 199900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (25, 1767, 3, 44, 252900),
 (26, 1888, 2, 79, 255000),
 (27, 1604, 3, 13, 242900),
 (28, 1962, 4, 53, 259900),
 (29, 3890, 3, 36, 573900),
 (30, 1100, 3, 60, 249900),
 (31, 1458, 3, 29, 464500),
 (32, 2526, 3, 13, 469000),
 (33, 2200, 3, 28, 475000),
 (34, 2637, 3, 25, 299900),
 (35, 1839, 2, 40, 349900),
 (36, 1000, 1, 5, 169900),
 (37

QUIZ: What would `DELETE FROM houses` do?

## Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [38]:
# ANSWER
c.execute('SELECT * FROM houses WHERE bdrms = 4 ').fetchall()

[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900)]

**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [39]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1827.5555555555557, 322892.51851851854),
 (4, 2399.733333333333, 373553.13333333336),
 (5, 4239.0, 659450.0)]

# Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [40]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

## Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [41]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('../DATA/housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [42]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

Run a query to get the average price of each house size from this table:

In [44]:
#ANSWER
c.execute('SELECT bdrms, AVG(price) as avg_price FROM houses_pandas GROUP BY bdrms').fetchall()

[(1, 169900.0),
 (2, 280866.6666666667),
 (3, 326403.92),
 (4, 377449.78571428574),
 (5, 699900.0)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

# Reference: SQL Syntax 


## SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

## WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

## AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

# Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

## Read CSV into DataFrame

In [45]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [46]:
# Reading CSV to Dataframe
orders = pd.read_csv('../DATA/P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('../DATA/P12-OrderBreakdown.csv', encoding = 'utf-8')

In [67]:
orders_break_down.to_csv('obd.csv')

In [54]:
orders_break_down

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies
...,...,...,...,...,...,...
8042,ES-2014-3458802,"Ikea Stackable Bookrack, Traditional",0.0,245.0,2,Furniture
8043,IT-2014-3715679,"BIC Pencil Sharpener, Fluorescent",0.5,30.0,2,Office Supplies
8044,IT-2014-3715679,"Avery Binder Covers, Recycled",0.5,23.0,4,Office Supplies
8045,IT-2014-3715679,"StarTech Phone, Red",0.5,108.0,3,Technology


## Replace Space with Underscore in Column Names

In [49]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [50]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

## Check DataTypes 

In [51]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [52]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

In [66]:
len(orders_break_down['order_id'].unique())

4117

## Merge two dataframes into one

In [72]:
left = orders_break_down
right = orders
orders_final = pd.merge(left,right , on='order_id', how='left')
orders_final

,order_id,product_name,discount,sales,quantity,category,order_date,customer_name,country
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies,2011-01-04,Grant Thornton,United Kingdom
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies,2011-01-04,Grant Thornton,United Kingdom
...,...,...,...,...,...,...,...,...,...
8042,ES-2014-3458802,"Ikea Stackable Bookrack, Traditional",0.0,245.0,2,Furniture,2014-12-31,John Grady,United Kingdom
8043,IT-2014-3715679,"BIC Pencil Sharpener, Fluorescent",0.5,30.0,2,Office Supplies,2014-12-31,Jim Kriz,Netherlands
8044,IT-2014-3715679,"Avery Binder Covers, Recycled",0.5,23.0,4,Office Supplies,2014-12-31,Jim Kriz,Netherlands
8045,IT-2014-3715679,"StarTech Phone, Red",0.5,108.0,3,Technology,2014-12-31,Jim Kriz,Netherlands


In [73]:
orders_final.isnull().sum()

order_id         0
product_name     0
discount         0
sales            0
quantity         0
category         0
order_date       0
customer_name    0
country          0
dtype: int64

## Save these two dataframes as a table in sqlite

In [82]:
#convert merged dataframe to a sqlite table
orders_final.to_sql('orders_final', con = conn, if_exists = 'replace', index = False)

#convert original 2 dataframes to sqlite tables
orders.to_sql('orders', con = conn, if_exists = 'replace', index = False)
orders_break_down.to_sql('orders_break_down', con = conn, if_exists = 'replace', index = False)


## Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [85]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT * 
        FROM orders
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=conn)
orders_head_10

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [86]:
# Select Number of Orders for Each Customer
query = """
        SELECT
        customer_name,
        COUNT(DISTINCT(order_id)) as no_orders
        FROM orders_final
        GROUP BY 1
        """

no_orders_customer = sql.read_sql(query, con=conn)
no_orders_customer


,customer_name,no_orders
0,Aaron Bergman,11
1,Aaron Hawkins,7
2,Aaron Smayling,12
3,Adam Bellavance,4
4,Adam Hart,8
...,...,...
787,Xylona Preis,2
788,Yana Sorensen,5
789,Yoseph Carroll,11
790,Zuschuss Carroll,9


## Select Number of Customers for Each Country

In [87]:
# Select Number of Customers for Each Country

query = """
        SELECT
        country,
        COUNT(DISTINCT(customer_name)) as no_customers
        FROM orders_final
        GROUP BY 1
        """

no_customers_country = sql.read_sql(query, con=conn)
no_customers_country


,country,no_customers
0,Austria,125
1,Belgium,66
2,Denmark,28
3,Finland,33
4,France,568
5,Germany,505
6,Ireland,48
7,Italy,351
8,Netherlands,172
9,Norway,37


## Select discount, sales, quantity for Each Order from orders_break_down Table

In [92]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table

query = """
        SELECT
        order_id,
        SUM(discount) AS ttl_discount,
        (sales * quantity) AS ttl_sales,
        SUM(quantity) AS ttl_qty
        FROM orders_final
        GROUP BY 1
        """

dsq_order = sql.read_sql(query, con=conn)
dsq_order


,order_id,ttl_discount,ttl_sales,ttl_qty
0,ES-2011-1001989,0.1,2061.0,9
1,ES-2011-1010958,0.0,116.0,8
2,ES-2011-1012469,0.0,444.0,3
3,ES-2011-1043483,0.0,2464.0,20
4,ES-2011-1058269,0.1,26.0,12
...,...,...,...,...
4112,IT-2014-5966070,0.5,428.0,4
4113,IT-2014-5975833,0.2,1312.0,5
4114,IT-2014-5984498,0.5,83.0,1
4115,IT-2014-5989338,0.1,14427.0,9


##  Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [93]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table

query = """
        SELECT
        order_id,
        SUM(discount) AS ttl_discount,
        (sales * quantity) AS ttl_sales,
        SUM(quantity) AS ttl_qty,
        (sales * quantity - discount) AS ttl_price
        FROM orders_final
        GROUP BY 1
        """

dsqp_order = sql.read_sql(query, con=conn)
dsqp_order


,order_id,ttl_discount,ttl_sales,ttl_qty,ttl_price
0,ES-2011-1001989,0.1,2061.0,9,2060.9
1,ES-2011-1010958,0.0,116.0,8,116.0
2,ES-2011-1012469,0.0,444.0,3,444.0
3,ES-2011-1043483,0.0,2464.0,20,2464.0
4,ES-2011-1058269,0.1,26.0,12,26.0
...,...,...,...,...,...
4112,IT-2014-5966070,0.5,428.0,4,427.5
4113,IT-2014-5975833,0.2,1312.0,5,1311.9
4114,IT-2014-5984498,0.5,83.0,1,82.5
4115,IT-2014-5989338,0.1,14427.0,9,14426.9


## Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [96]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100

query = """
        SELECT
        order_id,
        (sales * quantity - discount) AS ttl_price
        FROM orders_final
        WHERE (sales * quantity - discount) > 100
        GROUP BY 1
        
        """

order_100 = sql.read_sql(query, con=conn)
order_100


,order_id,ttl_price
0,ES-2011-1001989,2060.9
1,ES-2011-1010958,116.0
2,ES-2011-1012469,444.0
3,ES-2011-1043483,2464.0
4,ES-2011-1058269,2991.9
...,...,...
3654,IT-2014-5965314,239.5
3655,IT-2014-5966070,427.5
3656,IT-2014-5975833,1311.9
3657,IT-2014-5989338,14426.9


## Select All Customer And The Product They Have Bought

In [97]:
# Select All Customer And The Product They Have Bought
query = """
        SELECT
        customer_name,
        product_name
        FROM orders_final        
        """

cust_items = sql.read_sql(query, con=conn)
cust_items



,customer_name,product_name
0,Eugene Moren,"Enermax Note Cards, Premium"
1,Dorothy Dickinson,"Dania Corner Shelving, Traditional"
2,Arthur Prichep,"Binney & Smith Sketch Pad, Easy-Erase"
3,Grant Thornton,"Boston Markers, Easy-Erase"
4,Grant Thornton,"Eldon Folders, Single Width"
...,...,...
8042,John Grady,"Ikea Stackable Bookrack, Traditional"
8043,Jim Kriz,"BIC Pencil Sharpener, Fluorescent"
8044,Jim Kriz,"Avery Binder Covers, Recycled"
8045,Jim Kriz,"StarTech Phone, Red"


## Select Number of 'Furniture' Orders For Each Country 

In [99]:
# Select Number of 'Furniture' Orders For Each Country

query = """
        SELECT
        country,
        COUNT(DISTINCT(order_id)) as furniture_orders
        FROM orders_final
        WHERE category = 'Furniture'
        GROUP BY 1
        """

furniture_country = sql.read_sql(query, con=conn)
furniture_country



,country,furniture_orders
0,Austria,33
1,Belgium,19
2,Denmark,8
3,Finland,14
4,France,260
5,Germany,224
6,Ireland,10
7,Italy,117
8,Netherlands,63
9,Norway,6


## Select Number of 'Furniture' Orders For The Country Denmark

In [100]:
# Select Number of 'Furniture' Orders For The Country Denmark

# Select Number of 'Furniture' Orders For Each Country

query = """
        SELECT
        country,
        COUNT(DISTINCT(order_id)) as furniture_orders
        FROM orders_final
        WHERE category = 'Furniture' AND
        country = 'Denmark'
        GROUP BY 1
        """

furniture_denmark = sql.read_sql(query, con=conn)
furniture_denmark


,country,furniture_orders
0,Denmark,8


## Select Total Sales With Discount and Without Discount for Each Country

In [110]:
# Select Total Sales With Discount and Without Discount for Each Country

query = """
        SELECT
        country,
        COUNT(CASE WHEN discount > 0 THEN 1 END) as 'Sales_Discount',
        COUNT(CASE WHEN discount == 0 THEN 1 END) as 'Sales_Without_Discount',
        COUNT(product_name) AS total_sales_lines
        FROM orders_final
        GROUP BY 1
        """

disc_no_disc_country = sql.read_sql(query, con=conn)
disc_no_disc_country

,country,Sales_Discount,Sales_Without_Discount,total_sales_lines
0,Austria,0,264,264
1,Belgium,0,135,135
2,Denmark,60,0,60
3,Finland,0,64,64
4,France,854,1062,1916
5,Germany,599,1041,1640
6,Ireland,100,0,100
7,Italy,285,694,979
8,Netherlands,393,0,393
9,Norway,0,70,70


## Select Total Quantity, Total Sales for Each Country

In [113]:
# Select Total Quantity, Total Sales for Each Country

query = """
        SELECT
        country,
        SUM(sales * quantity) AS total_sales,
        SUM(quantity) AS total_quantity
        FROM orders_final
        GROUP BY 1
        """

sales_qty_country = sql.read_sql(query, con=conn)
sales_qty_country

,country,total_sales,total_quantity
0,Austria,391694.0,973
1,Belgium,229829.0,532
2,Denmark,34617.0,204
3,Finland,82001.0,201
4,France,3108592.0,7329
5,Germany,2411483.0,6179
6,Ireland,75724.0,392
7,Italy,1177921.0,3612
8,Netherlands,379714.0,1526
9,Norway,99878.0,261


## Select Top 3 Country Based on Sales

In [115]:
# Select Top 3 Country Based on Sales

# Select Total Quantity, Total Sales for Each Country

query = """
        SELECT
        country,
        SUM(sales * quantity) AS total_sales
        FROM orders_final
        GROUP BY 1
        ORDER BY 2 DESC
        LIMIT 3
        """

sales_country_top = sql.read_sql(query, con=conn)
sales_country_top

,country,total_sales
0,France,3108592.0
1,Germany,2411483.0
2,United Kingdom,2161947.0


## Select Bottom 3 Country Based On Quantities

In [116]:
# Select Bottom 3 Country Based On Quantities

# Select Top 3 Country Based on Sales

# Select Total Quantity, Total Sales for Each Country

query = """
        SELECT
        country,
        SUM(sales * quantity) AS total_sales
        FROM orders_final
        GROUP BY 1
        ORDER BY 2
        LIMIT 3
        """

sales_country_low = sql.read_sql(query, con=conn)
sales_country_low

,country,total_sales
0,Denmark,34617.0
1,Ireland,75724.0
2,Finland,82001.0


## Select Average Sales By Categroy For The Country 'France'

In [117]:
# Select Average Sales By Categroy For The Country 'France'

query = """
        SELECT
        category,
        avg(sales * quantity) AS avg_sales
        FROM orders_final
        WHERE country = 'France'
        GROUP BY 1
    
        """

sales_france = sql.read_sql(query, con=conn)
sales_france

,category,avg_sales
0,Furniture,3091.789298
1,Office Supplies,847.214460
2,Technology,2956.544041


## Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [124]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest

# Select Average Sales By Categroy For The Country 'France'

query = """
        SELECT
        country,
        category,
        avg(sales * quantity) AS avg_sales
        FROM orders_final
        GROUP BY 1,2
        ORDER BY avg_sales DESC
        LIMIT 1
    
        """

sales_avg_highest = sql.read_sql(query, con=conn)
sales_avg_highest

,country,category,avg_sales
0,Switzerland,Technology,6487.846154


# JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

#  ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax

© 2020 Institute of Data